<img src='http://imgur.com/1ZcRyrc.png' style='float: left; margin: 20px; height: 55px'>


# DSI-SG-42 Capstone Project:
### Silent Echoes: From Hand Waves to Written Phrases

# 2.1 Preprocessing of Video Files

Video files comes in different resolutions and fps values. We will have to process the files so that they are consistent for the model.

In this notebook, we will be cropping the videos so that the clip will only show the action that represents the word. Following that, we will also be resizing the clips into a standardized format of `30 fps` and with a video dimension of `512 x 512 pixels`. 

In [1]:
# import libraries
import os
import shutil
import logging
import cv2
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
from collections import defaultdict

# Configure logging to write to a file
log_dir = ('../log_files')
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file = os.path.join(log_dir, 'preprocessing.log')

# Setup Logger
logger = logging.getLogger()  # Get the root logger
for handler in logger.handlers:  # Remove all old handlers
    logger.removeHandler(handler)


logging.basicConfig(
    level=logging.INFO,
    format = '%(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()  
    ]
)


# set display settings
%matplotlib inline
pd.set_option('display.width', 100000)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

## 2.1 Import Data

We will import the various csv files to get information of the start and end frames of the sign action.


In [2]:
# Recalling first 3 words that we will be modeling upon

first_three = ['please', 'sorry', 'hello']

In [3]:
# import train dataset
train_df = pd.read_csv('../data/msasl_train.csv')

print(train_df.head()) # inspect dataframe

  clean_text                                            file  label  height  width    fps  start   end                                  cleaned_url
0      hello  American Sign Language Learning the basics.mp4      0     360    640  29.97   1874  1903  https://www.youtube.com/watch?v=yG09SQB0Hds
1      hello                        Unit 1 Vocabulary(2).mp4      0     360    640  29.97    280   362  https://www.youtube.com/watch?v=7YYB3BEoksc
2      hello                        Unit 1 Vocabulary(2).mp4      0     360    640  29.97    250   277  https://www.youtube.com/watch?v=7YYB3BEoksc
3      hello                        ASL Vocabulary hello.mp4      0     360    640  29.97    110   150  https://www.youtube.com/watch?v=9jUuFHB2m4M
4      hello                        ASL Vocabulary hello.mp4      0     360    640  29.97     30    80  https://www.youtube.com/watch?v=9jUuFHB2m4M


In [4]:
# import test dataset
test_df = pd.read_csv('../data/msasl_test.csv')

print(test_df.head()) # inspect dataframe

  clean_text                               file  label  height  width    fps  start  end                                  cleaned_url
0      hello        ASL1 Class 48 New SIGNS.mp4      0     720   1280  29.97     84  150  https://www.youtube.com/watch?v=kVIt-qr5P2s
1      hello  ASL Top ASL Signs for Servers.mp4      0     360    360  29.97     11   51  https://www.youtube.com/watch?v=FBimL8-ND3E
2      hello       ASL3 Class 23 Vocabulary.mp4      0     720   1280  29.97     92  124  https://www.youtube.com/watch?v=eLpHf9bfdsI
3      hello        ASL1 Class 29 New SIGNS.mp4      0     720   1280  29.97    106  136  https://www.youtube.com/watch?v=H3eHAKpTF70
4      hello  ASL Top ASL Signs for Servers.mp4      0     360    360  29.97    101  161  https://www.youtube.com/watch?v=FBimL8-ND3E


In [5]:
# import validation dataset
val_df = pd.read_csv('../data/msasl_val.csv')

print(val_df.head()) # inspect dataframe

  clean_text                                              file  label  height  width    fps  start    end                                  cleaned_url
0      hello                                        Unit 1.mp4      0     360    640  29.97  12647  12830  https://www.youtube.com/watch?v=BEKUHzwnVO8
1      hello         Mr Miseners ASL lesson 1 vocab review.mp4      0     720   1280  30.00   2149   2263  https://www.youtube.com/watch?v=kZuJalsv_z0
2      hello                               Social Work Voc.mp4      0     720   1280  30.00      6     39  https://www.youtube.com/watch?v=V8DTIPK4QlU
3      hello                     ASL I - Unit 1 Vocabulary.mp4      0     720   1280  24.00   2240   2316  https://www.youtube.com/watch?v=2VB3WN8adyM
4      hello  Greetings  Answers in American Sign Language.mp4      0     360    640  29.97     14     73  https://www.youtube.com/watch?v=pD4QabDQr6M


In [6]:
# import wlasl dataset

wlasl_df = pd.read_csv('../data/wlasl.csv')

print(wlasl_df.head()) # inspect dataframe

  clean_text    file  fps  start   end
0      hello  27177_   25      1    -1
1      hello  27175_   25      1    -1
2      hello  27171_   25   1297  1343
3      hello  70017_   25   4842  4923
4      hello  68236_   25      1    36


All the datasets have been imported and looking good to continue. From this point on, we will be cropping the video to the timestamps given or the targeted frames. 

To standardize all video files, we will proceed to resize the video to `512 x 512 pixels` and limit the frames per second to `30` to ensure consistency. 

As all the videos from the WL-ASL videos are already cropped we will leave it and continue to crop the videos from the MS-ASL.

Below is an example of a video file from WL-ASL. 

<video controls style = 'width: 65%; height: auto;'>
    <source src='../videos/wlasl_data/hello/27172_.mp4' type='video/mp4'>
</video>


## 2.2 Video Cropping

We need to clean the video files to clearly show **only** the signed action from a longer video clip from the Microsoft-ASL dataset videos that were downloaded from YouTube. Hence, we will create a function that crops the video according to the stipulated `start` and `end` frames. Followed by another function that will loop through each folder to crop the videos.

In [7]:
# define a crop function for a single video

def crop(video_path, start, stop, output_path):
    '''
    Crops a video and saves it to a new file.

    Args:
    video_path: Path to the source video file.
    start_sec: Start time of the desired clip in seconds.
    stop_sec: End time of the desired clip in seconds.
    output_path: Path to save the cropped video file.
    '''

    # Open the source video
    cap = cv2.VideoCapture(video_path)
    
    # break logic if the clip cannot be opened
    if not cap.isOpened():
        # print('Error opening video file: File not Available')
        logging.error('Error opening video file: File not Available %s', video_path)

        return

    # Get frames per second (fps) of the source video
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Set the starting and stopping frames
    start_frame = int(start)
    stop_frame = int(stop)

    # Set video codec and create a VideoWriter object to write the video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # video file is in mp4 format
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # obtain the width of the clip
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # obtain the height of the clip
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height)) # write the file

    # Set the starting position of the video
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # while loop to crop the file with the above conditions
    while True:

        # read the clip
        ret, frame = cap.read()
        if not ret:
            break

        # Get the current frame number and check if it is within the range
        current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)

        # break condition after the preassigned end time of action
        if current_frame > stop_frame:
            break

        # Write the frame to the output video
        out.write(frame)

    # Release everything when job is finished
    cap.release()
    out.release()
    #print(f'{video_path}: Video has been cropped and saved.')
    logging.info('Video has been cropped and saved. %s', video_path) # write log file

        

# define a function to crop all files within the same folder (action) 
def get_crop_video(word_label: str, save_folder:str, dataframe: pd.DataFrame):
    '''
    Crops videos based on a label and a DataFrame containing video information.

    Args:
    word_label: The label to filter videos by
    save_folder: The datset which the video will be found - train_data, test_data, val_data
    dataframe: A pandas DataFrame containing columns like 'file', 'start_time',
                'end_time', and 'clean_text'
    '''

    # temporary dataframe of target word/action
    label_df = dataframe[dataframe['clean_text'] == word_label]

    # Track the count of how many times each file is processed
    file_count = defaultdict(int)
    
    FOLDER_PATH = os.path.join('../videos/', save_folder ,  word_label) # folder of where the videos are stored
    OUTPUT_PATH = os.path.join(FOLDER_PATH,  'cropped_videos') # output of the video
    os.makedirs(OUTPUT_PATH, exist_ok=True)  # Create a output folder and ensure the output directory exists
    

    # Loop through videos and crop them
    for index, row in label_df.iterrows():
        video_path = os.path.join(FOLDER_PATH, row['file']) # get the video path
        start = row['start']
        stop = row['end']
        base, extension = os.path.splitext(row['file'])  # Separate filename base and extension

        # Generate new filename with underscore and incrementing number
        new_filename = f"{base}_{file_count[row['file']] + 1}{extension}" # assign a new video name
        output_path = os.path.join(OUTPUT_PATH, new_filename) # new video path

        # Increment processing count for the current file
        file_count[row['file']] += 1

        # intiate crop
        crop(video_path, start, stop, output_path)

        
    
    return 'All video files have been cropped'

Only Microsoft-ASL video files downloaded from YouTube requires video cropping. The WL-ASL files have frames that show the signed action, so they will not be part of the function call to crop the video.

In [8]:
# crop the videos in a single for-loop for all 3 folders and all actions

for vid in first_three:
    get_crop_video(vid, 'train_data', train_df)
    get_crop_video(vid, 'test_data', test_df)
    get_crop_video(vid, 'val_data', val_df)
    

INFO - Video has been cropped and saved. ../videos/train_data\please\Examples of Baby Sign Language.mp4
INFO - Video has been cropped and saved. ../videos/train_data\please\Examples of Baby Sign Language.mp4
INFO - Video has been cropped and saved. ../videos/train_data\please\Making Conversation Vocabulary  ASL - American Sign Language.mp4
INFO - Video has been cropped and saved. ../videos/train_data\please\Basic Sign Language for Caregivers of the DeafHard of Hearing.mp4
ERROR - Error opening video file: File not Available ../videos/train_data\please\ASL Vocabulary (Spring 2018).mp4
ERROR - Error opening video file: File not Available ../videos/train_data\please\Permission.mp4
INFO - Video has been cropped and saved. ../videos/train_data\please\Basic Sign Language for Caregivers of the DeafHard of Hearing.mp4
INFO - Video has been cropped and saved. ../videos/train_data\please\ASL Level 1 Unit 1 Vocabulary.mp4
ERROR - Error opening video file: File not Available ../videos/train_data\p

At the moment, WL-ASl dataset is separate from MS-ASL dataset, we will combing the video files from the WL-ASl into the train data folder which is from MS-ASl so that when we extract the keypoints from the video files, it can be done together. 

In [9]:
# Using a for-loop to move the videos from the WL-ASL dataset to another sub-folder titled 'cropped_videos' to combine training data

# recalling the actions we will be using
first_three = ['please', 'sorry', 'hello']

# the folder directories of WL-ASL and the destination folder
WLASL_PATH = '../videos/wlasl_data/' # main folder path for WL-ASL
DST_PATH = '../videos/train_data/'   # destination path for train data

# initiate for loop in the source firectory
for folder in os.listdir(WLASL_PATH):

    # only access the folder names in the list of words
    if folder in first_three: 
        # print(folder) # debug
        SOURCE_FOLDER = os.path.join(WLASL_PATH, folder ) # updated file path with folder names
        DST_FOLDER = os.path.join(DST_PATH, folder , 'cropped_videos') # destination path of where it should be copied to

        # nested loop for each video file inside each word folder
        for video in os.listdir(SOURCE_FOLDER):
            # print(video) # debug
            try:
                # make a copy of the video file to the newly made folder
                SOURCE_PATH = os.path.join(SOURCE_FOLDER, video ) # source file location

                # print(SOURCE_PATH) # debug
                # print(DST_FOLDER) # debug
                
                shutil.copy(SOURCE_PATH, DST_FOLDER) # creates a copy of the file

            except Exception as e:
                continue


## 2.3 Resize and adjust FPS


As the all videos, including WL-ASL, have different fps and are different dimensions, we will keep it standardized and adjust the frames-per-second (fps) to `30` and with an aspect ratio of `512 x 512 pixels`.

In [10]:
# define a function to resize a single video to the pre-determined fps and image size

def resize_video_file(input_video_path, output_video_path, fps=30, target_size=(512, 512)):
    '''
    Resizes a video to a specific size and frame rate.

    Args:
    input_video_path: Path to the input video file.
    output_video_path: Path to save the resized video file.
    fps: Target frames per second for the output video (default: 30).
    target_size: Target width and height for the resized video frames (default: (512, 512)).
    '''

    # Open the video file for reading
    cap = cv2.VideoCapture(input_video_path)

    # Check if the clip cannot be opened
    if not cap.isOpened():
        #print('Error: Cannot open video file ', input_video_path)
        logging.error('Error: Cannot open video file. %s', input_video_path)
        return

    # define the codec to be used for writing the file
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # write output video file with the above conditions
    out = cv2.VideoWriter(output_video_path, fourcc, fps, target_size)

    # initialize the start counter for number of processed frames
    processed_frames = 0

    # while the clip is read
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # resize the clip to the target size
        resized_frame = cv2.resize(frame, target_size)

        # write the resized frame to the output video file
        out.write(resized_frame)

        # break logic once 60 frames have been read
        if processed_frames == 60:
            break
        # increment the counter once each frame is read
        processed_frames += 1

    # release video capture object
    cap.release()
    # release video writer object
    out.release()

    #print(f'Number of frames processed: {processed_frames}') # debug
    #print('Video processing complete.') # debug
    logging.info('Number of frames processed: %s', processed_frames) # write logging file
    logging.info('Video processing complete.') # write logging file
    



# create a function to resize all files within the same folder (action)
def resize_class_videos(label:str, save_folder:str):
    '''
    Resizes all videos in a specific label's 'cropped_videos' directory.

    Args:
    label: The label of the video category to process.
    save_folder: The folder of the dataset - train_data, test_data, val_data
    '''

    # common path 
    GENERAL_PATH = '../videos/'

    # the folder of source video files
    LABEL_PATH = os.path.join(GENERAL_PATH, save_folder + '/' + label + '/cropped_videos/')

    # the destination folder of where the resized files will be saved
    OUTPUT_DIR = os.path.join('../videos/', save_folder + '/output/'  + label + '/')

    # create a destination folder and check if it exists
    os.makedirs(OUTPUT_DIR, exist_ok=True)  

    # intialize counter for file naming
    count = 1

    # loop each source folder
    for vid in os.listdir(LABEL_PATH):

        # video file path
        VIDEO_PATH = os.path.join(LABEL_PATH, vid)
        # print('Resizing: ', vid) # debug
        logging.info('Video file resized: %s', vid) # write into logging file

        # Ensure the output filename does not conflict and has the correct extension, and rename the resized video file
        OUT_PATH = os.path.join(OUTPUT_DIR, f'{label}_{count}.mp4')

        # call on the resize_video_file function defined above
        resize_video_file(VIDEO_PATH, OUT_PATH)
        count += 1 # increment counter

    return 'All video files have been resized'

In [11]:
# Resize the all video files with one loop to 30 fps and with an aspect ratio of 512 x 512 pixels

for vid in first_three:
    resize_class_videos(vid, 'train_data')
    resize_class_videos(vid, 'test_data')
    resize_class_videos(vid, 'val_data')
    

INFO - Video file resized: 43217_.mp4
INFO - Number of frames processed: 60
INFO - Video processing complete.
INFO - Video file resized: 43218_.mp4
INFO - Number of frames processed: 60
INFO - Video processing complete.
INFO - Video file resized: 43219_.mp4
INFO - Number of frames processed: 60
INFO - Video processing complete.
INFO - Video file resized: 43220_.mp4
INFO - Number of frames processed: 60
INFO - Video processing complete.
INFO - Video file resized: 43222_.mp4
INFO - Number of frames processed: 44
INFO - Video processing complete.
INFO - Video file resized: 43226_.mp4
INFO - Number of frames processed: 60
INFO - Video processing complete.
INFO - Video file resized: 69434_.mp4
INFO - Number of frames processed: 60
INFO - Video processing complete.
INFO - Video file resized: ASL Class 02-08-10_1.mp4
INFO - Number of frames processed: 60
INFO - Video processing complete.
INFO - Video file resized: ASL Level 1 Unit 1 Vocabulary_1.mp4
INFO - Number of frames processed: 60
INFO 

We will obtain the number of video files we have after cropping and resizing the video clips

In [12]:
# List of folders to count files in
folder_list = ['train_data', 'test_data', 'val_data']

# Function to count files in a folder
def count_files(folder_path):
    file_count = defaultdict(int)
    for file_name in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file_name)):
            file_count[''] += 1  # Counting files in the root folder
    return file_count

# Base path where the folders are located
FILE_PATH = '../videos/'

# Iterate through each action and folder to count files
for action in first_three:
    for folder in folder_list:
        # Construct the full path to the folder
        FOLDER_PATH = os.path.join(FILE_PATH, folder, action, 'cropped_videos')
        
        # Count the files in the folder
        file_count = count_files(FOLDER_PATH)
        
        # Print the count of files in the folder
        print(f'Files in {folder} {action}: {file_count[""]}')
    print('-' * 30)

Files in train_data please: 29
Files in test_data please: 3
Files in val_data please: 3
------------------------------
Files in train_data sorry: 18
Files in test_data sorry: 1
Files in val_data sorry: 10
------------------------------
Files in train_data hello: 17
Files in test_data hello: 10
Files in val_data hello: 5
------------------------------


---
## 2.4 Data Augmentation

As the amount of video files are not sufficient for a robust model, we will augment the video files through:

- flipping horizontally
- reducing the brightness
- adjusting the contrast
- shearing left
- shearing right
- gaussian blur

### 2.4.1 Flip the video horizontally

Horizontally flipping the videos increases the diversity of the training data by effectively doubling the amount of data available without changing the semantics of the video. It also encourages the model to learn features from another viewpoint, like in signed actions, flipping the video essentially changes the signing hand. This would help improve its generalization ability. 

In [13]:
# Function to flip a video horizontally
def flip_video_horizontally(input_video_path, output_video_path):
    '''
    Flip the video file horizontally

    Args:
    input_video_path: path of the video file
    output_video_path: path of the destination folder to write to
    '''
    # Open the input video file
    cap = cv2.VideoCapture(input_video_path)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    
    # Loop through each frame in the video
    while True:
        ret, frame = cap.read() # read the frame from the video
        if not ret: # if cannot read frame, break loop
            break

        # Flip the frame horizontally
        flipped_frame = cv2.flip(frame, 1)
        # write the flipped frame to the output video
        out.write(flipped_frame)

    # Release video capture object
    cap.release()
    # release video write object
    out.release()
    # print("Video flipped and saved to", output_video_path) # debug
    

# function to augment all video clips within the folder and action
def augment_by_flip(label:str, save_folder:str):
    '''
    Flip videos horizontally in a given folder by passing
    the following arguments

    Args:
    label: The label of the video category to process.
    save_folder: the folder name: 'train_data', 'test_data', 'val_data'
    '''
    # General path of video files
    GENERAL_PATH = '../videos/'
    
    # Construct  the path to the folder containing the labeled videos
    LABEL_PATH = os.path.join(GENERAL_PATH, save_folder + '/output/' + label + '/')

    # Define the output directory where flipped videos will be saved
    OUTPUT_DIR = os.path.join('../videos/', save_folder + '/output/'  + label + '/videos_to_annotate/')

    # create a destination folder and check if it exists
    os.makedirs(OUTPUT_DIR, exist_ok=True)  

    # intialize counter for file naming
    count = 1

    # get a list of video files in the labelled folder
    video_files = os.listdir(LABEL_PATH)
    
    # only read video files and not the created nested folder
    video_files = [f for f in video_files if f.endswith(('.mp4'))]
    
    # loop through each video in the video folder
    for vid in video_files:
        
        # construct the full path to the input video
        VIDEO_PATH = os.path.join(LABEL_PATH, vid)
        # print('Flipping: ', vid) # debug
        logging.info('Video augmented by flipping: %s', vid) # write to logging file
        

        # Ensure the output filename does not conflict and has the correct extension
        OUT_PATH = os.path.join(OUTPUT_DIR, f'{label}_flipped_{count}.mp4')

        # call the function to flip the video horizontally
        flip_video_horizontally(VIDEO_PATH, OUT_PATH)

        count += 1 # increment the count
       
    return 'All video files have been augmented by flipping horizontally'



### 2.4.2 Decrease Video Brightness

This simulates scenarios with varying lighting conditions, making the model robust to changes in illumination, it also prevents the model from overfitting to a specific lighting conditions present in the training data. Most of the videos in the dataset are in controlled environment with adequate lighting. 

In [14]:
# Function to decrease the brightness of the video
def make_video_darker(input_video_path, output_video_path, brightness_reduction=30):

    '''
    Decrease the brightness of the video

    Args:
    input_video_path: path of the video file
    output_video_path: path of the destination folder to write to
    brightness_reduction: amount to decrease brightness, default: 30
    '''

     # Open the input video file
    cap = cv2.VideoCapture(input_video_path)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    
    # Loop through each frame in the video
    while True:
        ret, frame = cap.read() # read the frame from the video
        if not ret: # if cannot read frame, break loop
            break

        # Convert to HSV to adjust brightness
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv) # hue, saturation, value

        # reduce brightness
        v = v.astype(np.float32)
        v = v * (1 - brightness_reduction / 100.0)  # reduce brightness
        v = np.clip(v, 0, 255).astype(np.uint8) # ensure pixel values are within 0-255

        # merge the HSV components back into one image
        final_hsv = cv2.merge((h, s, v))
        darker_frame = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
        out.write(darker_frame) # write the reduced brightness frame to the output video

    # Release video capture object
    cap.release()
    # release video write object
    out.release()
    # print("Video darkened and saved to", output_video_path) # debug


# define a function to decrease brightness by action and folder
def augment_by_darken(label:str, save_folder:str):
    '''
    Reduce brightness in a given folder by passing
    the following arguments

    Args:
    label: The label of the video category to process.
    save_folder: the folder name: 'train_data', 'test_data', 'val_data'
    '''

    # General path of video files
    GENERAL_PATH = '../videos/'

    # Construct  the path to the folder containing the labeled videos
    LABEL_PATH = os.path.join(GENERAL_PATH, save_folder + '/output/' + label + '/')

    # Define the output directory where videos will be saved
    OUTPUT_DIR = os.path.join('../videos/', save_folder + '/output/'  + label + '/videos_to_annotate/')

    # create a destination folder and check if it exists
    os.makedirs(OUTPUT_DIR, exist_ok=True)  

    # intialize counter for file naming
    count = 1

    # only read video files and not the created nested folder
    video_files = [f for f in os.listdir(LABEL_PATH) if f.endswith('.mp4')]
    
    # loop through each video in the video folder
    for vid in video_files:

        # construct the full path to the input video
        VIDEO_PATH = os.path.join(LABEL_PATH, vid)
        #print('Darkening: ', vid) # debug
        logging.info('Video augmented by reducing brightness: %s', vid) # write to logging file

        # Ensure the output filename does not conflict and has the correct extension
        OUT_PATH = os.path.join(OUTPUT_DIR, f'{label}_darkened_{count}.mp4')

        # call the function to decrease the brightness of the video clip
        make_video_darker(VIDEO_PATH, OUT_PATH)
        
        count += 1 # increment the count
        

    return f'All video files have been augmented by darkening the video'

### 2.4.3 Adjust Video Contrast

Changing the contrast can enhance or diminish the differences in intensity between pixels, which can bring out or supress certain features in a video. This can make the model more resilien to variations in contrast that may occur in the real-world scenarios.

In [15]:
# Function to decrease the contrast of the video
def change_video_contrast(input_video_path, output_video_path, contrast_adjustment):
    '''
    Flip the video file horizontally

    Args:
    input_video_path: path of the video file
    output_video_path: path of the destination folder to write to
    contrast_adjustment: factor to increase contrast a postive number decreases the contrast,
                                                     a negative number increases the contrast
    '''

    # Open the input video file
    cap = cv2.VideoCapture(input_video_path)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    # Loop through each frame in the video
    while True:
        ret, frame = cap.read() # read the frame from the video
        if not ret: # if cannot read frame, break loop
            break

        # increase contrast
        f = (1.0 - contrast_adjustment / 100.0)  # calculate contrast factor

        # change the contrast
        frame = cv2.addWeighted(frame, f, frame, 0, 128*(1-f)) 

        # write the adjusted contrast frame to the output video
        out.write(frame)

    # Release video capture object
    cap.release()

    # release video write object
    out.release()
    # print("Video contrast decreased and saved to", output_video_path) # debug



# function to augment all video clips within the folder and action
def augment_by_contrast(label:str, save_folder:str, contrast_adjustment: float):
    '''
    increase the contrast in a given folder by passing
    the following arguments

    Args:
    label: The label of the video category to process.
    save_folder: the folder name: 'train_data', 'test_data', 'val_data'
    '''

    # General path of video files
    GENERAL_PATH = '../videos/'

    # Construct  the path to the folder containing the labeled videos
    LABEL_PATH = os.path.join(GENERAL_PATH, save_folder + '/output/' + label + '/')

    # Define the output directory where videos will be saved
    OUTPUT_DIR = os.path.join('../videos/', save_folder + '/output/'  + label + '/videos_to_annotate/')

    # create a destination folder and check if it exists
    os.makedirs(OUTPUT_DIR, exist_ok=True)  # Ensure the output directory exists

    # intialize counter for file naming
    count = 1

    # get a list of video files in the labelled folder
    video_files = [f for f in os.listdir(LABEL_PATH) if f.endswith('.mp4')]

    # name logic to prevent conflicting naming of video file
    if contrast_adjustment > 0:
        contrast = 'decrease_contrast'
    elif contrast_adjustment < 0:
        contrast = 'increase_contrast'
    else:
        print('please choose a contrast greater than or less than 0')
    
    # loop through each video in the video folder
    for vid in video_files:
        
        # construct the full path to the input video
        VIDEO_PATH = os.path.join(LABEL_PATH, vid)
        # print('Reducing Contrast: ', vid) # debug
        logging.info('Video augmented by reducing the contrast: %s', vid) # write to logging file

        # Ensure the output filename does not conflict and has the correct extension
        OUT_PATH = os.path.join(OUTPUT_DIR, f'{label}_{contrast}_{count}.mp4')

        # call the function to adjust the contrast of the video
        change_video_contrast(VIDEO_PATH, OUT_PATH, contrast_adjustment)

        count += 1 # increment the count
        

    return f'All video files have been augmented by reducing contrast'

### 2.4.4 Augment by Shearing

Shearing left or right introduces distortion that simulates the effects of perspective changes or camera movements, making the model more robust to such variations. This encourages the model to learn the signed words from different angles or viewpoints, improving its ability to recognize objects from diverse perspectives.

In [16]:
# Function to flip shear a video

def shear_video(input_video_path, output_video_path, shear_factor):
    '''
    Shear video clip horizontally

    Args:
    input_video_path: path of the video file
    output_video_path: path of the destination folder to write to
    shear_factor: a float number, positive value will be sheared to the right
                                  negative value will be sheared to the left
    '''

    # Open the input video file
    cap = cv2.VideoCapture(input_video_path)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    fps = cap.get(cv2.CAP_PROP_FPS) # get fps
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # get frame width
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # get frame height

    # Output video writer
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Loop through each frame in the video
    while True:
        ret, frame = cap.read() # read the frame from the video
        if not ret: # if cannot read frame, break loop
            break

        # Transformation matrix for shearing
        M = np.array([
            [1, shear_factor, 0],
            [0, 1, 0]
        ], dtype=float)

        # Apply the shearing transformation
        sheared_frame = cv2.warpAffine(frame, M, (width + int(shear_factor * height), height))

        # Since the image might get cut off on the right, we need to resize it back to original
        sheared_frame = cv2.resize(sheared_frame, (width, height))

        # Write the transformed frame
        out.write(sheared_frame)

    # Release video capture object
    cap.release()
    # release video write object
    out.release()
    # print("Video sheared and saved to", output_video_path) # debug


# function to augment all video clips within the folder and action
def augment_by_shearing(label:str, save_folder:str, shear_factor: float):
    '''
    Shear the videos in a given folder by passing
    the following arguments

    Args:
    label: The label of the video category to process.
    save_folder: the folder name: 'train_data', 'test_data', 'val_data'
    shear_factor: the amount to shear with positive shear to the right, and negative shear to the left
    '''

    # General path of video files
    GENERAL_PATH = '../videos/'

    # Construct  the path to the folder containing the labeled videos
    LABEL_PATH = os.path.join(GENERAL_PATH, save_folder + '/output/' + label + '/')

    # Define the output directory where videos will be saved
    OUTPUT_DIR = os.path.join('../videos/', save_folder + '/output/'  + label + '/videos_to_annotate/')

    # create a destination folder and check if it exists
    os.makedirs(OUTPUT_DIR, exist_ok=True)  # Ensure the output directory exists

    # intialize counter for file naming
    count = 1
    
    # get a list of video files in the labelled folder
    video_files = [f for f in os.listdir(LABEL_PATH) if f.endswith('.mp4')]

    # name logic to prevent conflicting naming of video file
    if shear_factor > 0:
        shear = 'shear_positive'
    elif shear_factor < 0:
        shear = 'shear_negative'
    else:
        print('please choose a shear factor greater than or less than 0')
    
    # loop through each video in the video folder
    for vid in video_files:
        
        # construct the full path to the input video
        VIDEO_PATH = os.path.join(LABEL_PATH, vid)
        # print('Shearing: ', vid) # debug
        logging.info('Video augmented by shearing: %s', vid) # write into logging file

        # Ensure the output filename does not conflict and has the correct extension
        OUT_PATH = os.path.join(OUTPUT_DIR, f'{label}_{shear}_{count}.mp4')

        # call the function to shear the video
        shear_video(VIDEO_PATH, OUT_PATH, shear_factor)
        count += 1 # increment the count
        

    return f'All video files have been augmented by shearing'


### 2.4.5 Gaussian Blur

Gaussian blur smooths out fine details in the video, which can help the model focus on more prominent features and reduce sensitivity to noise. This would mimic the effect of motion blur that may occur due to camera or the signer's motion, making the model more robust.

In [17]:
# function to apply Gaussian blur to the video
def blur_video(input_video_path, output_video_path, kernel_size=(5, 5)):
    """
    Applies Gaussian blur to each frame of the video.
    
    Args:
    input_video_path (str): Path to the input video file.
    output_video_path (str): Path to the output video file.
    kernel_size (tuple): Size of the kernel used for the Gaussian blur.
    """
    # Open the input video file
    cap = cv2.VideoCapture(input_video_path)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS) # get the fps
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # get the frame width
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # get the frame height

    # Output video writer
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Loop through each frame in the video
    while True:
        ret, frame = cap.read() # read the frame from the video
        if not ret: # if cannot read frame, break loop
            break

        # Apply Gaussian blur to the frame
        blurred_frame = cv2.GaussianBlur(frame, kernel_size, 0)

        # Write the blurred frame to the output video
        out.write(blurred_frame)

    # Release video capture object
    cap.release()
    # release video write object
    out.release()
    # print("Video blurred and saved to", output_video_path) # debug


# function to augment all video clips within the folder and action
def augment_by_blur(label:str, save_folder:str):
    '''
    Apply Gaussian Blur to the videos in the folder directory
    Args:
    label: The label of the video category to process.
    save_folder: the folder name: 'train_data', 'test_data', 'val_data'
    '''

    # General path of video files
    GENERAL_PATH = '../videos/'

    # Construct  the path to the folder containing the labeled videos
    LABEL_PATH = os.path.join(GENERAL_PATH, save_folder + '/output/' + label + '/')

    # Define the output directory where videos will be saved
    OUTPUT_DIR = os.path.join('../videos/', save_folder + '/output/'  + label + '/videos_to_annotate/')

    # create a destination folder and check if it exists
    os.makedirs(OUTPUT_DIR, exist_ok=True)  # Ensure the output directory exists

    # intialize counter for file naming
    count = 1

    # only read video files and not the created nested folder
    video_files = [f for f in os.listdir(LABEL_PATH) if f.endswith('.mp4')]
    
    # loop through each video in the video folder
    for vid in video_files:
        
        # construct the full path to the input video
        VIDEO_PATH = os.path.join(LABEL_PATH, vid)
        # print('Applying Gaussian blur: ', vid) # debug
        logging.info('Video augmented by applying Gaussian Blur: %s', vid) # write to logging file

        # Ensure the output filename does not conflict and has the correct extension
        OUT_PATH = os.path.join(OUTPUT_DIR, f'{label}_Gblur_{count}.mp4')

        # call the function to apply Gaussian blue
        blur_video(VIDEO_PATH, OUT_PATH)
        
        count += 1 # increment the count
        

    return f'All video files have been augmented by applying Gaussian Blur'

### 2.4.6 Apply augmentation to the video files

With the above defined function to augment the video files, we are now able to use a single loop to apply all the augmentation function to the video files.

In [18]:
folder_list = ['train_data', 'test_data', 'val_data']

for vid in first_three:
    for folder in folder_list:
        augment_by_flip(vid,folder) # flip the clip horizontally
        augment_by_darken(vid,folder) # reduce the brightness
        augment_by_contrast(vid,folder, 30) # decrease the contrast
        augment_by_contrast(vid,folder, -30) # increase the contrast
        augment_by_shearing(vid,folder, 0.1) # shear to the right
        augment_by_shearing(vid,folder, -0.1) # shear to the left
        augment_by_blur(vid,folder) # apply gaussian blur


INFO - Video augmented by flipping: please_1.mp4
INFO - Video augmented by flipping: please_10.mp4
INFO - Video augmented by flipping: please_11.mp4
INFO - Video augmented by flipping: please_12.mp4
INFO - Video augmented by flipping: please_13.mp4
INFO - Video augmented by flipping: please_14.mp4
INFO - Video augmented by flipping: please_15.mp4
INFO - Video augmented by flipping: please_16.mp4
INFO - Video augmented by flipping: please_17.mp4
INFO - Video augmented by flipping: please_18.mp4
INFO - Video augmented by flipping: please_19.mp4
INFO - Video augmented by flipping: please_2.mp4
INFO - Video augmented by flipping: please_20.mp4
INFO - Video augmented by flipping: please_21.mp4
INFO - Video augmented by flipping: please_22.mp4
INFO - Video augmented by flipping: please_23.mp4
INFO - Video augmented by flipping: please_24.mp4
INFO - Video augmented by flipping: please_25.mp4
INFO - Video augmented by flipping: please_26.mp4
INFO - Video augmented by flipping: please_27.mp4
IN

## 2.5 Organization of files

The above augmentation steps create a nested folder within and we will have to reorganize the files so that they are combined together into a single folder for efficient extraction of keypoints. We will move all the augmented files into the folder where the original files are located. 

We will use `hello` in the `train_data` folder as an example. The folder path for the original resized and cropped video files for `hello` are found in `'../videos/train_data/hello/cropped_videos/`. While the augmented video files are found in the nested folder '../videos/train_data/hello/cropped_videos/`videos_to_annotate`. Hence, we will move all the augmented video files into the `cropped_videos` where the unaltered videos are and delete the empty `videos_to_annotate` folder. 

In [19]:
# create a function to move a single video file to the folder destination
def move_video(source_file_path, destination_directory):
    '''
    Moves a single video file to the specified destination directory.

    Args:
    source_file_path (str): The path to the source video file.
    destination_directory (str): The path to the destination directory where the video should be moved.
    '''
    # Ensure the destination directory exists
    os.makedirs(destination_directory, exist_ok=True)
    
    # Define the destination file path
    video_file = os.path.basename(source_file_path)
    destination_file_path = os.path.join(destination_directory, video_file)

    # Move the video file
    shutil.move(source_file_path, destination_file_path)
    # print(f'Moved {video_file} to {destination_directory}') # debug
    logging.info(f'Moved {video_file} to {destination_directory}') # write to logging file
    

# function to move the video files to the respective destination folder and delete the nested folder
def copy_to_annotate(label, train_test_val_type):
    GENERAL_PATH = '../videos/'
    SOURCE_PATH = os.path.join(GENERAL_PATH, train_test_val_type, 'output', label, 'videos_to_annotate')
    OUTPUT_DIR = os.path.join(GENERAL_PATH, train_test_val_type, 'output', label)

    # Ensure the output directory exists
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Check if the source directory exists
    if os.path.exists(SOURCE_PATH) and os.path.isdir(SOURCE_PATH):
        # Iterate through each video file in the source directory
        for vid in os.listdir(SOURCE_PATH):
            VID_PATH = os.path.join(SOURCE_PATH, vid)
            if os.path.isfile(VID_PATH):  # Check if it is a file
                move_video(VID_PATH, OUTPUT_DIR)

        # Once all files have been moved, delete the source directory
        shutil.rmtree(SOURCE_PATH)
        print(f'Deleted directory {SOURCE_PATH}')
    else:
        print(f'Source directory {SOURCE_PATH} does not exist.')

    return 'All original resized video files have been moved and the source directory deleted'

In [20]:
# initiate the organization steps

folder_list = ['train_data', 'test_data', 'val_data']

for vid in first_three:
    for folder in folder_list:
        copy_to_annotate(vid, folder)
    print('-' * 30)

INFO - Moved please_darkened_1.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_10.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_11.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_12.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_13.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_14.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_15.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_16.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_17.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_18.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_19.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_2.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_20.mp4 to ../videos/train_data\output\please
INFO - Moved please_darkened_21.mp4 to .

Deleted directory ../videos/train_data\output\please\videos_to_annotate
Deleted directory ../videos/test_data\output\please\videos_to_annotate
Deleted directory ../videos/val_data\output\please\videos_to_annotate
------------------------------


INFO - Moved sorry_decrease_contrast_5.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_decrease_contrast_6.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_decrease_contrast_7.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_decrease_contrast_8.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_decrease_contrast_9.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_flipped_1.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_flipped_10.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_flipped_11.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_flipped_12.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_flipped_13.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_flipped_14.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_flipped_15.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_flipped_16.mp4 to ../videos/train_data\output\sorry
INFO - Moved sorry_flipped_17.mp4

Deleted directory ../videos/train_data\output\sorry\videos_to_annotate
Deleted directory ../videos/test_data\output\sorry\videos_to_annotate
Deleted directory ../videos/val_data\output\sorry\videos_to_annotate
------------------------------


INFO - Moved hello_darkened_10.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_11.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_12.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_13.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_14.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_15.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_16.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_17.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_2.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_3.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_4.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_5.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_6.mp4 to ../videos/train_data\output\hello
INFO - Moved hello_darkened_7.mp4 to ../videos/train_data\output\hell

Deleted directory ../videos/train_data\output\hello\videos_to_annotate
Deleted directory ../videos/test_data\output\hello\videos_to_annotate


INFO - Moved hello_shear_negative_4.mp4 to ../videos/val_data\output\hello
INFO - Moved hello_shear_negative_5.mp4 to ../videos/val_data\output\hello
INFO - Moved hello_shear_positive_1.mp4 to ../videos/val_data\output\hello
INFO - Moved hello_shear_positive_2.mp4 to ../videos/val_data\output\hello
INFO - Moved hello_shear_positive_3.mp4 to ../videos/val_data\output\hello
INFO - Moved hello_shear_positive_4.mp4 to ../videos/val_data\output\hello
INFO - Moved hello_shear_positive_5.mp4 to ../videos/val_data\output\hello


Deleted directory ../videos/val_data\output\hello\videos_to_annotate
------------------------------
